In [23]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
import folium
from folium import plugins

In [2]:
data=pd.read_excel('./Hospitals and Beds in Lebanon - 18 April 2021.xlsx')

In [3]:
data['Year']=pd.to_datetime(data['Year'],format="%Y",errors='coerce')

In [4]:
data

,Id,Hospital,Year,Location,Add,Address,Number of Beds
0,1,Hopital Dr S. Serhal,1982-01-01,Metn/Baabda,"Lebanon,El Rabieh","El Rabieh, Rue Michael Serhal",60
1,2,Hotel Dieu de France,1923-01-01,Beirut,Achrafieh - Beirut - Lebanon,Hotel Dieu de France - Bvd. Alfred Naccache - ...,430
2,3,Clinique Dr. Rizk,1925-01-01,Beirut,Rue Zahar - Achrafieh - Beyrouth - Liban,Rue Zahar - Achrafieh - Beyrouth - Liban,160
3,4,Makassed,1930-01-01,Beirut,"Lebanon,Tarik El Jadida",Tarik El Jadida,200
4,5,Al-Rassoul Al-Aazam hospital,1988-01-01,Beirut,Bourj Al-Barajneh - Airport Road - Beirut,Bourj Al-Barajneh - Airport Road - Beirut,240
...,...,...,...,...,...,...,...
163,169,Bshamoun Specialty Hospital,NaT,Chouf/Aley,"Lebanon,Bchamoun",NaN,32
164,170,Beirut Eye Specialty Hospital,NaT,Beirut,"Lebanon,Ashrafieh","Achrafieh, Rue Hôtel Dieu",20
165,171,Dar El Chifaa,NaT,North Lebanon,"Lebanon,Tripoli",NaN,50
166,172,Dar Al Zahraa,NaT,North Lebanon,"Lebanon, Tripoli",Tripoli,0


In [26]:
vmin, vmax = data['Number of Beds'].min(), data['Number of Beds'].max()

In [27]:
norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
cmap = matplotlib.cm.get_cmap('GnBu') # green to blue color way

In [31]:
traces=[]
for xd, yd in zip(data['Location'].unique(), data['Number of Beds']):
        median = np.median(yd) # find the median
        color = 'rgb' + str(cmap(norm(median))[0:3]) # normalize
        
        traces.append(go.Box(
            y=yd,
            name=xd,
            boxpoints='all',
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor=color,      # add the box plot color
            marker=dict(
                size=2,
            ),
            line=dict(width=1)
        ))

ValueError: 
    Invalid value of type 'builtins.int' received for the 'y' property of box
        Received value: 60

    The 'y' property is an array that may be specified as a tuple,
    list, numpy array, or pandas Series

In [42]:
layout = go.Layout(
    yaxis=dict(
        autorange=True,
        showgrid=False,
        zeroline=True,
        dtick=5,
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
        zerolinecolor='rgb(255, 255, 255)',
        zerolinewidth=2,
    ),
    margin=dict(
        l=40,
        r=30,
        b=80,
        t=100,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

In [59]:
import plotly.express as px
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Box(x=data['Location'],y=data['Number of Beds'],boxpoints='all',line=dict(color='rgb(0, 0, 102)'
),jitter=0.5,whiskerwidth=0.2))
fig.update_layout( plot_bgcolor='rgb(204,229,255)')
fig.update_layout(title_text="Number of beds across Lebanon")
fig.show()

In [65]:
import requests
import urllib.parse
address = 'Lebanon '
url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
response = requests.get(url).json()
print(response[0]["lat"])
print(response[0]["lon"])

33.8750629
35.843409


In [66]:
def get_add_lat(x):
    address = str(x)
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
    response = requests.get(url).json()
    print(response[0]["lat"])
#     print(response[0]["lon"])

In [67]:
def get_lat(x):
    address=str(x)
    url='https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
#     response=requests.get(url).json()
    return requests.get(url).json()[0]['lat']
    

In [68]:
def get_long(x):
    address=str(x)
    url='https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
#     response=requests.get(url).json()
    return requests.get(url).json()[0]['lon']

In [69]:
data['Add_lat']=data['Add'].apply(lambda x: get_lat(x))

In [70]:
data['Add_long']=data['Add'].apply(lambda x: get_long(x))

In [190]:
# m=folium.Map([33.8750629,35.843409])
# folium.TileLayer('Stamen Terrain').add_to(m)
# folium.TileLayer('Stamen Toner').add_to(m)
# folium.TileLayer('Stamen Water Color').add_to(m)
# folium.TileLayer('cartodbpositron').add_to(m)
# folium.TileLayer('cartodbdark_matter').add_to(m)
# folium.LayerControl().add_to(m)
# for i in range(0,len(data)):
#     folium.Circle(
#       location=[data.iloc[i]['Add_lat'], data.iloc[i]['Add_long']],
#       popup=data.iloc[i]['Hospital'],
#       radius=float(data.iloc[i]['Number of Beds']*2),
#       color='crimson',
#       fill=True,
#       fill_color='green'
#    ).add_to(m)
# m

In [71]:
m=folium.Map([33.8750629,35.843409])
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Water Color').add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
folium.LayerControl().add_to(m)
for i in range(0,len(data)):
    folium.Marker(
      location=[data.iloc[i]['Add_lat'], data.iloc[i]['Add_long']],
      popup=data.iloc[i]['Hospital'],icon=folium.Icon(color='green',icon='glyphicon glyphicon-plus-sign')
   ).add_to(m)
m.save("hospitals.html")

In [186]:
# # convert to (n, 2) nd-array format for heatmap
# stationArr = pd.DataFrame([response[0]["lat"],response[0]["lon"]]).as_matrix()

# # plot heatmap
# m.add_children(plugins.HeatMap([response[0]["lat"],response[0]["lon"]], radius=15))
# m

In [72]:
from folium.plugins import HeatMap
m=folium.Map([33.8750629,35.843409])
df_copy = data.copy()
HeatMap(data=df_copy[['Add_lat', 'Add_long', 'Number of Beds']].groupby(['Add_lat', 'Add_long']).sum().reset_index().values.tolist(), radius=20).add_to(m)
m.save("heatmap.html")

In [698]:
data.to_excel('final_data.xlsx')

In [680]:
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent='younasspider@gmail.com')
# location = geolocator.geocode("Lebanon,El Rabieh")
# print(location.address)
# print((location.latitude, location.longitude))
# folium.Marker(location=[location.latitude,location.longitude],).add_to(m)
# m

In [681]:
# from folium.plugins import HeatMap
# df_copy = df.copy()
# base_map = generateBaseMap()
# HeatMap(data=df_copy[['pickup_latitude', 'pickup_longitude', 'count']].groupby(['pickup_latitude', 'pickup_longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)

In [682]:
# /Users/younes/Downloads/countries.geojson

In [683]:
# m = folium.Map([39.3710358,-84.2128058])
# m

In [684]:
# import selenium
# from selenium import webdriver
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.options import Options 
# option = webdriver.ChromeOptions()
# option.add_argument('headless')

# driver = webdriver.Chrome("/Users/younes/Desktop/Younes/booking_library/chromedriver",options=option)
# driver.get('https://www.latlong.net/')
# element = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.ID, "onetrust-accept-btn-handler")))
# # accept_er=driver.find_elements_by_xpath('/html/body/div[4]/div[3]/div/div[1]/div/div[2]/div/button[1]')
# element.click()
# location=WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.ID,"place")))
# # location=driver.find_elements_by_xpath('//*[@id="place"]')
# location.clear()
# location.send_keys('Lebanon')
# find_button=WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.ID,"btnfind")))
# # find_button=driver.find_elements_by_xpath('//*[@id="btnfind"]')
# find_button.click()
# latitude=WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.ID,"lat")))
# longitude=WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.ID,"lng")))
# # latitude=driver.find_elements_by_xpath('//*[@id="lat"]')
# # longitude=driver.find_elements_by_xpath('//*[@id="lng"]')
# print(latitude.get_attribute('value'))
# print(longitude.get_attribute('value'))


In [685]:
# import geopandas

In [686]:
# gdf = geopandas.GeoDataFrame(
#     data['Address'], geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

In [687]:
# import requests
# address='1600+Amphitheatre+Parkway'
# response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address='+str(address)+',+Mountain+View,+CA')

# resp_json_payload = response.json()

# print(resp_json_payload['results'][0]['geometry']['location'])